# Walmart : predict weekly sales

## Company's Description 📇

Walmart Inc. is an American multinational retail corporation that operates a chain of hypermarkets, discount department stores, and grocery stores from the United States, headquartered in Bentonville, Arkansas. The company was founded by Sam Walton in 1962.

## Project 🚧

Walmart's marketing service has asked you to build a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made. Such a model would help them understand better how the sales are influenced by economic indicators, and might be used to plan future marketing campaigns.

## Goals 🎯

The project can be divided into three steps:

- Part 1 : make an EDA and all the necessary preprocessings to prepare data for machine learning
- Part 2 : train a **linear regression model** (baseline)
- Part 3 : avoid overfitting by training a **regularized regression model**

## Scope of this project 🖼️

For this project, you'll work with a dataset that contains information about weekly sales achieved by different Walmart stores, and other variables such as the unemployment rate or the fuel price, that might be useful for predicting the amount of sales. The dataset has been taken from a Kaggle competition, but we made some changes compared to the original data. Please make sure that you're using **our** custom dataset (available on JULIE). 🤓

## Deliverable 📬

To complete this project, your team should: 

- Create some visualizations
- Train at least one **linear regression model** on the dataset, that predicts the amount of weekly sales as a function of the other variables
- Assess the performances of the model by using a metric that is relevant for regression problems
- Interpret the coefficients of the model to identify what features are important for the prediction
- Train at least one model with **regularization (Lasso or Ridge)** to reduce overfitting


In [47]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score

# EDA and preprocessing

## Raw dataset

In [48]:
# Import du dataset

Walmart = pd.read_csv("./Walmart_Store_sales.csv")
print(Walmart.shape)
display(Walmart.head())
display(Walmart.dtypes)

(150, 8)


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

In [49]:
# Statistiques basiques sur le dataset original

print("Number of rows : {}".format(Walmart.shape[0]))
print("Number of columns : {}".format(Walmart.shape[1]))

print("\nBasics statistics: ")
display(Walmart.describe(include="all"))

print("\nPercentage of missing values: ")
display(100 * Walmart.isnull().sum() / Walmart.shape[0])

print("\nNumber of duplicated lines: ")
print(Walmart.duplicated().sum())

Number of rows : 150
Number of columns : 8

Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64


Percentage of duplicated lines: 
0


Commentaires sur le preprocessing à faire :
1) drop columns
- pas de colonne avec "trop" de valeurs manquantes -> pas de colonne à dropper, sauf la colonne "Date" quand on l'aura remplacée par les colonnes year, month etc. (pour éviter la multicolinéarité)
2) drop rows
- missing values dans le target (Weekly_Sales) : à retirer
- outliers : il ne semble pas y en avoir ailleurs que dans Unemployment, mais à vérifier plus en détail 
3) Imputation 
- il faudra imputer les valeurs manquantes de toutes les colonnes, sauf Store qui n'a pas de NaN
4) Standardisation
- StandardScaler à faire pour toutes les colonnes quantitatives
5) Encoding
- colonne Holiday_Flag est un catégoriel : valeurs 0 ou 1 -> pas besoin de faire du OneHotEncoder
- colonne Store est un catégoriel : de 1 à 20 (avec un peu plus de grandes valeurs que de petites, puisque 2nd quartile à 9 au lieu de 10) -> on va effectuer un OneHotEncoder mais on pourrait envisager de les regrouper en moins de catégories

In [50]:
# Echelle de temps des échantillons

Walmart['Date'] = pd.to_datetime(Walmart["Date"], format = "%d-%m-%Y")
print("date minimale : ", Walmart['Date'].min())
print("date maximale : ", Walmart['Date'].max())

date minimale :  2010-02-05 00:00:00
date maximale :  2012-10-19 00:00:00


In [51]:
# colonne Date : 85 valeurs uniques sur les 132 non-NaN, est-ce une répartition uniforme ?

dict_datesuniques = {}
list_uniques = Walmart['Date'].unique()
list_uniques = list_uniques[~pd.isnull(list_uniques)] # to remove NaN=0
for unique in list_uniques:
    mask = Walmart['Date'] == unique
    dict_datesuniques[unique] = len(Walmart.loc[mask, :])

fig = px.histogram(list(dict_datesuniques.keys()), dict_datesuniques.values(), title="Number of repetitions for all unique dates")
fig.show()

px.bar(x=Walmart['Date'], title="Count of each unique date")

# Conclusion : il n'y a pas de date sur-représentée dans le dataset, on utilisera un KNN pour l'imputation des NaN

## Preprocessing

In [52]:
# drop des lignes sans target

Walmart.dropna(subset=["Weekly_Sales"], inplace=True)

Walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896


In [53]:
# définition de la limite des outliers (mean +/- 3 * std) pour les 4 variables quantitatives

data_describe = Walmart.describe(include="all")

col_X_quanti = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]

data_dict = {
    'mean': data_describe.loc["mean",col_X_quanti],
    'std': data_describe.loc["std",col_X_quanti]
}
df_stat = pd.DataFrame(data_dict)
df_stat['outliers_min'] = (df_stat['mean'] - 3*df_stat['std'])
df_stat['outliers_max'] = (df_stat['mean'] + 3*df_stat['std'])
df_stat.head()

,mean,std,outliers_min,outliers_max
Temperature,60.853967,18.514432,5.310671,116.397263
Fuel_Price,3.316992,0.47954,1.878371,4.755613
CPI,178.091144,40.243105,57.36183,298.820458
Unemployment,7.665582,1.619428,2.807297,12.523867


In [54]:
# vérification de la présence d'outliers :
# histogrammes des 4 features quantitatives, avec visualisation des limites mean +/- 3 * std

fig = make_subplots(rows = 4, cols = 1)

fig.add_trace(
    go.Histogram(
        x = Walmart["Temperature"], name = "Temperature"),
        row = 1,
        col = 1)
fig.add_shape(
    type="line",
    x0=df_stat.loc["Temperature","outliers_min"],
    y0=0,
    x1=df_stat.loc["Temperature","outliers_min"],
    y1=25,
    xref='x',
    yref="y",
    line=dict(
        color="Red",
        width=2
    ),
    row=1,
    col=1
)
fig.add_shape(
    type="line",
    x0=df_stat.loc["Temperature","outliers_max"],
    y0=0,
    x1=df_stat.loc["Temperature","outliers_max"],
    y1=25,
    xref='x',
    yref="y",
    line=dict(
        color="Red",
        width=2
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Histogram(
        x = Walmart["Fuel_Price"], name = "Fuel_Price"),
        row = 2,
        col = 1)
fig.add_shape(
    type="line",
    x0=df_stat.loc["Fuel_Price","outliers_min"],
    y0=0,
    x1=df_stat.loc["Fuel_Price","outliers_min"],
    y1=30,
    xref='x',
    yref="y",
    line=dict(
        color="Red",
        width=2
    ),
    row=2,
    col=1
)
fig.add_shape(
    type="line",
    x0=df_stat.loc["Fuel_Price","outliers_max"],
    y0=0,
    x1=df_stat.loc["Fuel_Price","outliers_max"],
    y1=30,
    xref='x',
    yref="y",
    line=dict(
        color="Red",
        width=2
    ),
    row=2,
    col=1
)

fig.add_trace(
    go.Histogram(
        x = Walmart["CPI"], name = "CPI"),
        row = 3,
        col = 1)
fig.add_shape(
    type="line",
    x0=df_stat.loc["CPI","outliers_min"],
    y0=0,
    x1=df_stat.loc["CPI","outliers_min"],
    y1=55,
    xref='x',
    yref="y",
    line=dict(
        color="Red",
        width=2
    ),
    row=3,
    col=1
)
fig.add_shape(
    type="line",
    x0=df_stat.loc["CPI","outliers_max"],
    y0=0,
    x1=df_stat.loc["CPI","outliers_max"],
    y1=55,
    xref='x',
    yref="y",
    line=dict(
        color="Red",
        width=2
    ),
    row=3,
    col=1
)

fig.add_trace(
    go.Histogram(
        x = Walmart["Unemployment"], name = "Unemployment"),
        row = 4,
        col = 1)
fig.add_shape(
    type="line",
    x0=df_stat.loc["Unemployment","outliers_min"],
    y0=0,
    x1=df_stat.loc["Unemployment","outliers_min"],
    y1=22,
    xref='x',
    yref="y",
    line=dict(
        color="Red",
        width=2
    ),
    row=4,
    col=1
)
fig.add_shape(
    type="line",
    x0=df_stat.loc["Unemployment","outliers_max"],
    y0=0,
    x1=df_stat.loc["Unemployment","outliers_max"],
    y1=22,
    xref='x',
    yref="y",
    line=dict(
        color="Red",
        width=2
    ),
    row=4,
    col=1
)

fig.update_layout(width=700, height=1000)

fig.show()

# conclusion : il y a bien des outliers uniquement dans la colonne Unemployement

In [55]:
# dropping des outliers 

Walmart_clean = Walmart.copy(deep=True)

for col in col_X_quanti:
    min = df_stat.loc[col,"outliers_min"]
    max = df_stat.loc[col,"outliers_max"]
    for row in Walmart.index:
        if Walmart.loc[row,col] < min or Walmart.loc[row,col] > max:
            Walmart_clean.drop(row, axis=0, inplace=True)

Walmart_clean.reset_index(inplace=True)
Walmart_clean.drop('index', axis=1, inplace=True)
print("Number of rows : {}".format(Walmart_clean.shape[0]))

# on a perdu 5 lignes supplémentaires, on est à 87% du dataset originel

Number of rows : 131


In [56]:
# Créer les colonnes year, month, day à partir de la colonne Date (qui a des NaN)

Walmart_clean['year'] = Walmart_clean['Date'].dt.year
Walmart_clean['month'] = Walmart_clean['Date'].dt.month
Walmart_clean['day'] = Walmart_clean['Date'].dt.day
Walmart_clean['weekday'] = Walmart_clean['Date'].dt.weekday
Walmart_clean.drop("Date", axis=1, inplace=True)

print(Walmart_clean['weekday'].value_counts()) 
# les relevés sont toujours faits les vendredis -> on drop la colonne weekday
Walmart_clean = Walmart_clean.drop("weekday", axis=1)

display(Walmart_clean.describe())

weekday
4.0    113
Name: count, dtype: int64


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day
count,131.000000,1.310000e+02,120.000000,117.000000,119.000000,120.000000,117.000000,113.000000,113.000000,113.000000
mean,9.938931,1.257990e+06,0.066667,60.405897,3.302908,180.175755,7.399427,2010.831858,6.274336,16.530973
std,6.228663,6.577463e+05,0.250490,18.466740,0.475435,39.723167,0.994117,0.822699,3.179869,8.238705
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000,2010.000000,1.000000,1.000000
25%,4.000000,5.842439e+05,0.000000,44.820000,2.824000,132.579257,6.664000,2010.000000,4.000000,10.000000
50%,9.000000,1.366396e+06,0.000000,61.790000,3.435000,197.655672,7.368000,2011.000000,6.000000,17.000000
75%,16.000000,1.809576e+06,0.000000,75.540000,3.708500,214.904838,8.099000,2012.000000,9.000000,24.000000
max,20.000000,2.771397e+06,1.000000,91.650000,4.170000,226.968844,9.524000,2012.000000,12.000000,31.000000


## EDA after preprocessing

In [57]:
print("\nPercentage of missing values Walmart: ")
display(100 * Walmart.isnull().sum() / Walmart.shape[0])

print("\nPercentage of missing values Walmart_clean: ")
display(100 * Walmart_clean.isnull().sum() / Walmart_clean.shape[0])


Percentage of missing values Walmart: 


Store            0.000000
Date            13.235294
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature     11.029412
Fuel_Price       8.823529
CPI              8.088235
Unemployment    10.294118
dtype: float64


Percentage of missing values Walmart_clean: 


Store            0.000000
Weekly_Sales     0.000000
Holiday_Flag     8.396947
Temperature     10.687023
Fuel_Price       9.160305
CPI              8.396947
Unemployment    10.687023
year            13.740458
month           13.740458
day             13.740458
dtype: float64

In [58]:
px.bar(Walmart_clean, x="month", y="Weekly_Sales", facet_col="year", facet_col_wrap=2)

In [59]:
px.histogram(Walmart_clean, "day", color="year", barmode="overlay")

In [135]:
px.histogram(Walmart, "Store", nbins=20)

In [134]:
px.histogram(Walmart_clean, "Store", nbins=20)

In [60]:
px.bar(Walmart_clean, x="Store", y="Weekly_Sales")

In [61]:
px.histogram(Walmart_clean, x="Temperature", y="Weekly_Sales")

In [62]:
px.histogram(Walmart_clean, x="Holiday_Flag", y="Weekly_Sales")

In [63]:
px.histogram(Walmart_clean, x="Fuel_Price", y="Weekly_Sales")

In [64]:
px.histogram(Walmart_clean, x="CPI", y="Weekly_Sales")

In [136]:
px.histogram(Walmart_clean, x="Unemployment", y="Weekly_Sales")

In [65]:
fig = px.scatter_matrix(Walmart_clean)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=1200, width = 1200)
fig.show()

In [66]:
corr_matrix = Walmart_clean.corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

Conclusions : 
- aucune feature ne présente une corrélation de type linéaire avec la target
- les colonnes *quantitatives* qui semblent le plus liées à Weekly_sales sont : CPI, Unemployement, Temperature
- le Store 12 a été supprimé en retirant les outliers (les 5 outliers étaient de ce store)

## Creation of X and Y datasets

In [67]:
# Separate target variable Y from features X

print("Separating labels from features...")
target_variable = "Weekly_Sales"

X = Walmart_clean.drop(target_variable, axis = 1)
Y = Walmart_clean.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    1572117.54
1    1807545.43
2    1244390.03
3    1644470.66
4    1857533.70
Name: Weekly_Sales, dtype: float64

X :
   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           NaN        59.61       3.045  214.777523         6.858   
1   13.0           0.0        42.38       3.435  128.616064         7.470   
2   11.0           0.0        84.57         NaN  214.556497         7.346   
3    6.0           0.0        78.89       2.759  212.412888         7.092   
4    4.0           0.0          NaN       2.756  126.160226         7.896   

     year  month   day  
0  2011.0    2.0  18.0  
1  2011.0    3.0  25.0  
2     NaN    NaN   NaN  
3  2010.0    5.0  28.0  
4  2010.0    5.0  28.0  


In [68]:
# Divide dataset Train set & Test set 

print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [69]:
# Define names of numeric/categorical columns

display(X.dtypes)
numeric_features = list(X.columns)
numeric_features.remove('Store')
numeric_features.remove('Holiday_Flag')
categorical_features = ["Store", "Holiday_Flag"]
print("numeric features:", numeric_features)
print("categorical features:", categorical_features)

Store           float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
year            float64
month           float64
day             float64
dtype: object

numeric features: ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'year', 'month', 'day']
categorical features: ['Store', 'Holiday_Flag']


In [70]:
# Pipelines to impute missing values, standardize quantitative features and use OHE on missing values

numeric_transformer = Pipeline(
    steps=[
        ("imputer", KNNImputer(n_neighbors=5)),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", KNNImputer(n_neighbors=5)), 
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [71]:
# Transforming X_train and X_test 

print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print("...Done.")
print(
    X_train[0:5]
)  
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test)  
print("...Done.")
print(
    X_test[0:5, :]
)  
print()

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
118    4.0           NaN        84.59       3.469  129.112500         5.644   
68     1.0           0.0        62.25       3.308  218.220509         7.866   
16    18.0           0.0        21.33       2.788  131.527903         9.202   
96    18.0           0.0        69.12       2.906  132.293936           NaN   
122    5.0           0.0        62.37         NaN  212.560411         6.768   

       year  month   day  
118  2011.0    7.0   8.0  
68   2011.0   11.0  18.0  
16      NaN    NaN   NaN  
96   2010.0    5.0  28.0  
122  2010.0   11.0  12.0  
...Done.
  (0, 0)	1.4376493924421494
  (0, 1)	0.32943014218256045
  (0, 2)	-1.3400305240541759
  (0, 3)	-1.7677757942410028
  (0, 4)	0.19112366352991209
  (0, 5)	0.19334690984647673
  (0, 6)	-1.1536424118724797
  (0, 9)	1.0
  (1, 0)	0.13065244125222247
  (1, 1)	-0.024085694357687964
  (1, 2)	1.0203743126966742
  (1, 3)	

In [72]:
# Creation of the dictionnary keeping trace of the different models' results

dict_results = {}

# Baseline : linear regression model

In [73]:
# Model Linear Regression

regressor = LinearRegression()
regressor.fit(X_train, Y_train)

Y_train_pred = regressor.predict(X_train)
Y_test_pred = regressor.predict(X_test)
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

dict_results['baseline'] = [r2_score(Y_train, Y_train_pred), r2_score(Y_test, Y_test_pred)]

R2 score on training set :  0.9726241276810833
R2 score on test set :  0.9439154012879177


In [74]:
# Cross-validation of the std on train

scores = cross_val_score(regressor, X_train, Y_train, cv=10)
avg = scores.mean()
std = scores.std()
print('Cross-validated accuracy : {}\nstandard deviation : {}'.format(avg, std))

# conclusion : on n'est pas en overfit, c'est dû à la petite taille du dataset

Cross-validated accuracy : 0.9366456417285182
standard deviation : 0.04166820378546107


In [101]:
# Analyzing the values of the model's coefficients

column_names = []
for name, pipeline, features_list in preprocessor.transformers_: 
    if name == 'num': 
        features = features_list 
    else: 
        features = pipeline.named_steps['encoder'].get_feature_names_out()
        features = list(features)
        features = np.char.replace(features, 'x0', features_list[0])
        features = np.char.replace(features, 'x1', features_list[1])
    column_names.extend(features) 
        
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])

feature_importance = coefs.sort_values(by = 'coefficients', key=abs)

fig = px.bar(feature_importance, orientation = 'h', height=700)
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} 
                 )
fig.show()

# Regularized regression model

In [137]:
# Ridge

print("Grid search Ridge...")
model_ridge = Ridge()
params = {
    'alpha': [0, 0.001, 0.005, 0.01, 0.05, 0.1] 
}
gridsearchR = GridSearchCV(model_ridge, param_grid = params, cv = 3) 
gridsearchR.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearchR.best_params_)
print("Best R2 score : ", gridsearchR.best_score_)

Grid search Ridge...
...Done.
Best hyperparameters :  {'alpha': 0}
Best R2 score :  0.9278682630508389


In [93]:
# Lasso

print("Grid search Lasso...")
model_lasso = Lasso()
params = {
    'alpha': [1, 10, 100, 400, 500, 600, 700, 800, 850, 875, 900, 925, 950, 1000] 
}
gridsearchL = GridSearchCV(model_lasso, param_grid = params, cv = 3) 
gridsearchL.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearchL.best_params_)
print("Best R2 score : ", gridsearchL.best_score_)

Grid search Lasso...
...Done.
Best hyperparameters :  {'alpha': 400}
Best R2 score :  0.9285467102001231


In [102]:
# Features selected by Lasso(900)

best_lasso = Lasso(alpha=900)
best_lasso.fit(X_train, Y_train)

coefs_lasso = pd.DataFrame(index = column_names, data = best_lasso.coef_.transpose(), columns=["Lasso900"])

feature_importance = coefs_lasso.sort_values(by = 'Lasso900', key=abs)
#display(feature_importance)

fig = px.bar(feature_importance, orientation = 'h', height=700)
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} 
                 )
fig.show()

### Conclusion

In [104]:
best_model_ridge = Ridge(alpha=0.05)
best_model_ridge.fit(X_train, Y_train)
Y_train_pred = best_model_ridge.predict(X_train)
Y_test_pred = best_model_ridge.predict(X_test)
dict_results['Ridge_05'] = [r2_score(Y_train, Y_train_pred), r2_score(Y_test, Y_test_pred)]

best_model_lasso = Lasso(alpha=900)
best_model_lasso.fit(X_train, Y_train)
Y_train_pred = best_model_lasso.predict(X_train)
Y_test_pred = best_model_lasso.predict(X_test)
dict_results['Lasso_900'] = [r2_score(Y_train, Y_train_pred), r2_score(Y_test, Y_test_pred)]

df_results = pd.DataFrame(dict_results, index=['R2_train', 'R2_test'])
display(df_results)

,baseline,Ridge_05,Lasso_900
R2_train,0.972624,0.972269,0.970882
R2_test,0.943915,0.946530,0.953546


In [107]:
# Analyzing the values of Lasso's coefficients

scores = cross_val_score(best_model_lasso, X_train, Y_train, cv=10)
avg = scores.mean()
std = scores.std()
print('Cross-validated accuracy : {}\nstandard deviation : {}'.format(avg, std))

coefs = pd.DataFrame(index = column_names, data = best_model_lasso.coef_.transpose(), columns=["coefficients"])

feature_importance = coefs.sort_values(by = 'coefficients', key=abs)
feature_importance

fig = px.bar(feature_importance, orientation = 'h', height=700)
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} 
                 )
fig.show()

Cross-validated accuracy : 0.9395224305221959
standard deviation : 0.04610312690012112


### Visualisation

In [131]:
# PCA: visualising the 3 better axes according to the target

from sklearn.decomposition import PCA

pca = PCA(n_components=3, random_state=0)
X_opt_train = pca.fit_transform(X_train.toarray())
print(f'Explained Variance ratio per PC: {pca.explained_variance_ratio_}')
print(f'Total explained variance ratio: {pca.explained_variance_ratio_.sum()}%')

X_opt_test = pca.transform(X_test.toarray())

best_model_lasso.fit(X_opt_train, Y_train)
print("score on test set: ", best_model_lasso.score(X_opt_test, Y_test))

df_pc = pd.DataFrame(X_opt_train)
df_test = pd.DataFrame(X_opt_test)
df_pc.columns = ["PC1", "PC2", "PC3"]
df_pc['Weekly_Sales'] = Y_train
fig = px.scatter_3d(df_pc, x="PC1", y="PC2", z="PC3", color="Weekly_Sales")
fig.show()

# components des 3 PC :
data = pd.DataFrame(pca.components_)
data.columns=column_names
print('PCA components for the first 3 axes:')
display(data)

#Associer les poids de PC1 aux noms des variables
pc1_components = pca.components_[0]
pc1_df = pd.DataFrame({
    'Variable': column_names,          # Noms des variables d'origine
    'PC1_Contribution': pc1_components  # Poids des variables dans PC1
})

#Trier par ordre décroissant d'importance
pc1_df = pc1_df.sort_values(by='PC1_Contribution', key=abs, ascending=False)
print("Weight of each feature in the first axis:")
display(pc1_df)

Explained Variance ratio per PC: [0.24736931 0.17491263 0.14375349]
Total explained variance ratio: 0.5660354311005934%
score on test set:  -0.011493036439371052


PCA components for the first 3 axes:


,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,Store_2.0,Store_3.0,Store_4.0,...,Store_13.0,Store_14.0,Store_15.0,Store_16.0,Store_17.0,Store_18.0,Store_19.0,Store_20.0,Holiday_Flag_0.2,Holiday_Flag_1.0
0,-0.296929,0.603607,-0.145018,-0.020605,0.645774,-0.299509,-0.112345,0.002362,-0.032181,-0.009629,...,0.016003,0.027721,0.020169,0.013942,0.001903,0.010286,0.016631,-0.004955,-0.006933,0.003594
1,-0.460351,-0.219379,-0.352485,0.636950,-0.250880,-0.350213,-0.049343,0.017281,-0.046387,-0.022297,...,0.018424,0.060733,0.014243,-0.020108,-0.013600,0.050140,0.050986,0.001648,0.011264,0.014055
2,0.048961,-0.001673,-0.568396,-0.132234,-0.046131,0.396845,-0.686512,-0.026183,0.002210,0.058062,...,0.069049,-0.037021,0.022100,-0.010322,0.077643,0.015120,0.013412,-0.029301,-0.030893,-0.022567


Weight of each feature in the first axis:


,Variable,PC1_Contribution
4,year,0.645774
1,Fuel_Price,0.603607
5,month,-0.299509
0,Temperature,-0.296929
2,CPI,-0.145018
6,day,-0.112345
12,Store_7.0,0.037433
8,Store_3.0,-0.032181
14,Store_9.0,-0.029302
18,Store_14.0,0.027721


# Conclusion et Perspectives

Conclusion : les 3 modèles se valent, le meilleur est le Lasso (alpha = 900) qui attribue un poids nul à un des Store (#19)

Tests effectués non montrés dans ce Notebook :
- régression linéaire sans la colonne Store : donne un R2 de 0 -> les stores sont indispensables pour les prédictions
- faire l'imputation des dates directement sur la colonne Date, qui contient une information plus précise/complète que les colonnes year/month/day mais cela oblige à faire le KNN avant d'avoir créé X_train et X_test, donc pas de différenciation fit_transform et simple transform -> cela change un peu les figures (autant de 2011 que de 2010, mois un peu plus équilibrés) mais ne change presque pas les résultats des modèles
- ne pas retirer les outliers de Unemployement puiqu'ils sont tous sur le Store 12 : les scores R2 sont plus faibles (~93%) et la baseline passe en overfit d'après la cross-validation

Perspectives :
- faire du feature engineering sur la colonne Store, pour ne pas avoir 20 colonnes après OHE, en faisant 2-4 catégories de stores (selon la target voire en combinaison avec d'autres features) 
- trouver les variables explicatives "cachées" entre les différents X, car au vu des pair-plots on voit de possibles clustering -> on peut créer des colonnes combinant variables quantitative et qualitative, et pour les duo de variables quantitatives on peut tester du feature engineering